<a href="https://colab.research.google.com/github/fumio125/ou_dip/blob/master/ou_dip_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [ ]:
# 先週のコードと異なり、周波数領域でのエイリアシングを防ぐために画像と同サイズの空間フィルタを作るようにした

def createFrequencyGaussian(sigma, img):
  # sigma: 対応する空間ガウシアンフィルタのsigma
  spatial_1d = cv2.getGaussianKernel(img.shape[0], sigma)   
  spatial_filter = np.matmul(spatial_1d, spatial_1d.T)  # 空間フィルタ
  # 空間フィルタをDFTして周波数フィルタを生成（他の方法もあるが、実装が楽なので）
  gauss_freq = cv2.dft(spatial_filter.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT) # 1ch目に実部、2ch目に虚数部
  gauss_freq += 1e-5  # 後段の0割り算を回避するため（本来は、逆フィルタの方で微小値を足すのが良い）

  # 最大値が1の周波数空間でのガウス型ローパスフィルタ
  re, im = cv2.split(gauss_freq)
  gauss_freq = cv2.magnitude(re, im)
  gauss_freq = gauss_freq / np.max(gauss_freq)

  return gauss_freq

In [ ]:
# 準備：画像の読み込みとフィルタの生成
sigma = 9 # 対応する空間ガウシアンフィルタのsigma

src = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み
#src = cv2.imread("pano_ref.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み
#src = cv2.imread("pano_ref_full.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み

print('Frequency filter:')
freq_filter = createFrequencyGaussian(sigma, src)  # 左上原点！
cv2_imshow(freq_filter  * 255) # 最大値が255になるようにして表示

In [ ]:
# 周波数フィルタによる画像劣化

# 画像をぼかす
src_freq = cv2.dft(src.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT) # 1ch目に実部、2ch目に虚数部 にしたい場合
blurred_freq = src_freq * np.dstack((freq_filter,freq_filter))  # 実部・虚数部両方に掛ける
blurred = cv2.idft(blurred_freq, flags=cv2.DFT_SCALE)[:,:,0]  # 逆フーリエ変換

# 微小なノイズを付加する（正規分布 sigma=0.5）
noise = np.random.normal(0.0, 0.5, blurred.shape)
blurred = blurred + noise # ここをコメントアウトすると、逆フィルタでもきれいに復元できるが。。。？

print("original image")
cv2_imshow(src)

print("degraded image")
cv2_imshow(blurred)

In [ ]:
# 逆フィルタによる画像復元
# 劣化画像: blurred, 点拡がり関数のフーリエ変換H: freq_filter

# inverse filter
inv_filter = 1.0/freq_filter
# cv2_imshow((inv_filter-1.0) * 255) # 逆フィルタを可視化したいとき

# inverse filterの適用
blurred_freq = cv2.dft(blurred.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT) # 1ch目に実部、2ch目に虚数部 にしたい場合
deblur_freq = blurred_freq * np.dstack((inv_filter,inv_filter))  # 実部・虚数部両方に掛ける
deblur = cv2.idft(deblur_freq, flags=cv2.DFT_SCALE)[:,:,0]  # 逆フーリエ変換

# 表示
print("reconstructed image")
cv2_imshow(deblur)

In [ ]:
# ウィーナフィルタによる画像復元
# 劣化画像: blurred, 点拡がり関数のフーリエ変換H: freq_filter

# wiener filter（ここを作成）
gamma = 
wiener_filter = 

# wiener filterの適用
blurred_freq = cv2.dft(blurred.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT) # 1ch目に実部、2ch目に虚数部 にしたい場合
deblur_freq = blurred_freq * np.dstack((wiener_filter,wiener_filter))  # 実部・虚数部両方に掛ける
deblur = cv2.idft(deblur_freq, flags=cv2.DFT_SCALE)[:,:,0]  # 逆フーリエ変換

# 表示
print("reconstructed image")
cv2_imshow(deblur)